In [334]:
from torch import nn
import torch

def replicate(layer,N):
    return nn.ModuleList([layer for _ in range(N)])

class Encode(nn.Module):

    def __init__(self,layer,N):
        super(Encode, self).__init__()
        self.layer=replicate(layer,N)
        self.N=N

    def forward(self,X,mask):
        for layer in self.layer:
            X=layer(X,mask)
        return X

In [291]:
class LayerNorm(nn.Module):

    def __init__(self,features,eps=1e-16):
        super(LayerNorm, self).__init__()
        self.a_2=nn.Parameter(torch.ones(features))
        self.b_2=nn.Parameter(torch.zeros(features))
        self.eps=eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


In [292]:
class AddNorm(nn.Module):
    def __init__(self, size, dropout_rate, eps=1e-6):
        super(AddNorm, self).__init__()
        
        # Layer normalization component
        self.norm = LayerNorm(size, eps=eps)
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, sublayer_output):
        "Apply residual connection followed by layer normalization"
        # Residual connection
        added_output = x + self.dropout(sublayer_output)
        
        # Layer normalization
        return self.norm(added_output)

In [293]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        d_model: the number of expected features in the input (required).
        d_ff: the number of features of the feedforward network model.
        dropout: the dropout value (default=0.1).
        """
        super(FeedForward, self).__init__()
        
        # Two linear layers with a ReLU activation in between
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [294]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [295]:
import torch.nn.functional as F
def attention(query,Q,K,V):
    #K transpose might cause a trpoouble if we have multi dimntioanmatrix
    scores=torch.matmul(Q,K.T)/math.sqrt(d_k)
    p_atten=scores.softmax(scores, dim=2)
    return torch.matmul(p_atten,V),p_atten

In [296]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, query, key,value,mask=None):
        batch_size = query.size(0)

        # Linear layers
        Q = self.query(query)
        K = self.key(key)
        V = self.value(value)

        # Split into multiple heads
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)

        # Scaled dot-product attention
        attention_scores = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.d_k**0.5
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_probs, V)

        # Concatenate heads and pass through final linear layer
        attention_output = attention_output.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_k * self.num_heads)
        output = self.fc_out(attention_output)

        return output

In [312]:
class Encoder(nn.Module):
    def __init__(self, size, FeedForward, Multi_Head_Attention,AddNorm):
        super(Encoder, self).__init__()
        self.self_attn = Multi_Head_Attention
        self.feed_forward = FeedForward
        self.AddNorm=AddNorm
        self.size = size

    def forward(self,x,mask):
        x=self.AddNorm(x, self.self_attn(x,x,x,mask))
        x=self.AddNorm(x, self.feed_forward(x))
        return x      

In [284]:
# src=torch.LongTensor([0,1,2,3,4,5,6,7,8,9])
# tgt=torch.LongTensor([0,1,2,3,4,5,6,7,8,9])

# model=Encoder_Decoder(
#         Encode(Encoder(512,FeedForward(512, 2048, dropout=0.1), MultiHeadAttention(512, 8),AddNorm(512, 0.1, eps=1e-6)),6),
#         Decode(Decoder(512, FeedForward(512, 2048, dropout=0.1), MultiHeadAttention(512, 8),MultiHeadAttention(512,8)),6),
#         nn.Sequential(Embedding(512,10),Positional_Encoding(512)),
#         nn.Sequential(Embedding(512,10),Positional_Encoding(512)),
#         Generator(512,10)
#     )
# A=model.src_embed(src)
# B=model.Encode(A)

In [262]:
# B=MultiHeadAttention(512, 8)
# B(A,A,A).shape


torch.Size([10, 1, 512])

In [350]:
#this might be a replicate
class Decode(nn.Module):

    def __init__(self,layer,N):
        super(Decode, self).__init__()
        self.layer=replicate(layer,N)
        self.N=N

    def forward(self,X,y,src_mask,tgt_mask):
        for layer in self.layer:
            X=layer(X,y,src_mask,tgt_mask)
        return X

In [351]:
class Decoder(nn.Module):
    def __init__(self, size, FeedForward, Self_Multi_Head_Attention,Encoder_Multi_Head_Attention):
        super(Decoder, self).__init__()
        self.self_attn = Self_Multi_Head_Attention
        self.feed_forward = FeedForward
        self.encoder_attention=Encoder_Multi_Head_Attention
        self.size = size

    def forward(self,x,m,src_mask,tgt_mask):
        x=self.AddNorm(x, self.self_attn(x,x,x,tgt_mask))
        x=self.AddNorm(x, self.encoder_attention(x,m,m,src_mask))
        x=self.AddNorm(x, FeedForward(x))
        return x   

In [300]:
class Embedding(nn.Module):
    def __init__(self, d_model,vocab):
        super(Embedding, self).__init__()
        self.embedding=nn.Embedding(vocab,d_model)

    def forward(self,x):
        return self.embedding(x)   

In [301]:
class Positional_Encoding(nn.Module):
    def __init__(self, d_model):
        super(Positional_Encoding, self).__init__()
        self.dmodel=d_model
        
    def forward(self,x):
        n=x.shape[0]
        div_term = torch.exp(torch.arange(0., self.dmodel, 2) * -(math.log(10000.0) / self.dmodel))

        positions = torch.arange(n).unsqueeze(1).float()
        div_term = div_term.unsqueeze(0)
        sin_vals = torch.sin(positions * div_term)
        cos_vals = torch.cos(positions * div_term)

        ZZ = torch.empty(n, self.dmodel)
        ZZ[:, 0::2] = sin_vals
        ZZ[:, 1::2] = cos_vals
        return x+ZZ

In [174]:
# def Positional_Encoding(Embeded):
#     n=Embeded.shape[0]
#     dmodel=Embeded.shape[1]
#     div_term = torch.exp(torch.arange(0., dmodel, 2) * -(math.log(10000.0) / dmodel))

#     # Expand dimensions for broadcasting
#     positions = torch.arange(n).unsqueeze(1).float()
#     div_term = div_term.unsqueeze(0)

#     # Calculate sin and cos values
#     sin_vals = torch.sin(positions * div_term)
#     cos_vals = torch.cos(positions * div_term)

#     # Interleave sin and cos values
#     ZZ = torch.empty(n, dmodel)
#     ZZ[:, 0::2] = sin_vals
#     ZZ[:, 1::2] = cos_vals

In [341]:
class Encoder_Decoder(nn.Module):
    def __init__(self, Encode,Decode,src_embed, tgt_embed, generator):
        super(Encoder_Decoder, self).__init__()
        self.Encode=Encode
        self.Decode=Decode
        self.src_embed=src_embed
        self.tgt_embed=tgt_embed
        self.generator=generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.Encode(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.Decode(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [303]:
class Generator(nn.Module):
    def __init__(self,d_model,tgt_vocab):
        super(Generator, self).__init__()
        self.linear=nn.Linear(d_model,tgt_vocab)

    def forward(sefl,x):
        x=self.linear(x)
        return log_softmax(self.proj(x), dim=-1)

In [352]:
def make_model(src_vocab,tgt_vocab,N=6,d_model=512,h=8,dropout=0.1,d_ff=2048):
    
    model=Encoder_Decoder(
        Encode(Encoder(d_model,FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),AddNorm(d_model,dropout, eps=1e-6)),N),
        Decode(Decoder(d_model, FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),MultiHeadAttention(d_model,h)),N),
        nn.Sequential(Embedding(d_model,src_vocab),Positional_Encoding(d_model)),
        nn.Sequential(Embedding(d_model,tgt_vocab),Positional_Encoding(d_model)),
        Generator(d_model,tgt_vocab)
    )
    for p in model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
    return model

In [353]:
test_model = make_model(11, 11, 2)
test_model.eval()
src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
src_mask = torch.ones(1, 1, 10)
memory = test_model.encode(src, src_mask)
ys = torch.zeros(1, 1).type_as(src)


for i in range(9):
    out = test_model.decode(
        memory, src_mask, embed_ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
    prob = test_model.generator(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data[0]
    ys = torch.cat(
        [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
    )

print("Example Untrained Model Prediction:", ys)



RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [16]:
from torch import nn
import torch
import math 
import torch.nn.functional as F

def replicate(layer,N):
    return nn.ModuleList([layer for _ in range(N)])

class Encode(nn.Module):

    def __init__(self,layer,N):
        super(Encode, self).__init__()
        self.layer=replicate(layer,N)
        self.N=N

    def forward(self,X,mask):
        for layer in self.layer:
            X=layer(X,mask)
        return X
    
class LayerNorm(nn.Module):

    def __init__(self,features,eps=1e-16):
        super(LayerNorm, self).__init__()
        self.a_2=nn.Parameter(torch.ones(features))
        self.b_2=nn.Parameter(torch.zeros(features))
        self.eps=eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
class AddNorm(nn.Module):
    def __init__(self, size, dropout_rate, eps=1e-6):
        super(AddNorm, self).__init__()
        
        # Layer normalization component
        self.norm = LayerNorm(size, eps=eps)
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, sublayer_output):
        "Apply residual connection followed by layer normalization"
        # Residual connection
        added_output = x + self.dropout(sublayer_output)
        
        # Layer normalization
        return self.norm(added_output)
    
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        d_model: the number of expected features in the input (required).
        d_ff: the number of features of the feedforward network model.
        dropout: the dropout value (default=0.1).
        """
        super(FeedForward, self).__init__()
        
        # Two linear layers with a ReLU activation in between
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, query, key,value,mask=None):
        batch_size = query.size(0)

        # Linear layers
        Q = self.query(query)
        K = self.key(key)
        V = self.value(value)

        # Split into multiple heads
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).permute(0, 2, 1, 3)

        # Scaled dot-product attention
        attention_scores = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.d_k**0.5
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_probs, V)

        # Concatenate heads and pass through final linear layer
        attention_output = attention_output.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_k * self.num_heads)
        output = self.fc_out(attention_output)

        return output
    
class Encoder(nn.Module):
    def __init__(self, size, FeedForward, Multi_Head_Attention,AddNorm):
        super(Encoder, self).__init__()
        self.self_attn = Multi_Head_Attention
        self.feed_forward = FeedForward
        self.AddNorm=AddNorm
        self.size = size

    def forward(self,x,mask):
        x=self.AddNorm(x, self.self_attn(x,x,x,mask))
        x=self.AddNorm(x, self.feed_forward(x))
        return x  
    
class Embedding(nn.Module):
    def __init__(self, d_model,vocab):
        super(Embedding, self).__init__()
        self.embedding=nn.Embedding(vocab,d_model)

    def forward(self,x):
        return self.embedding(x)   
    
class Positional_Encoding(nn.Module):
    def __init__(self, d_model):
        super(Positional_Encoding, self).__init__()
        self.dmodel=d_model
        
    def forward(self,x):
        n=x.shape[0]
        div_term = torch.exp(torch.arange(0., self.dmodel, 2) * -(math.log(10000.0) / self.dmodel))

        positions = torch.arange(n).unsqueeze(1).float()
        div_term = div_term.unsqueeze(0)
        sin_vals = torch.sin(positions * div_term)
        cos_vals = torch.cos(positions * div_term)

        ZZ = torch.empty(n, self.dmodel)
        ZZ[:, 0::2] = sin_vals
        ZZ[:, 1::2] = cos_vals
        return x+ZZ
    
def make_model(src_vocab,tgt_vocab,N=6,d_model=512,h=8,dropout=0.1,d_ff=2048):
    
    model=Encoder_Decoder(
        Encode(Encoder(d_model,FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),AddNorm(d_model,dropout, eps=1e-6)),N),
        Decode(Decoder(d_model, FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),MultiHeadAttention(d_model,h)),N),
        nn.Sequential(Embedding(d_model,src_vocab),Positional_Encoding(d_model)),
        nn.Sequential(Embedding(d_model,tgt_vocab),Positional_Encoding(d_model)),
        Generator(d_model,tgt_vocab)
    )
    for p in model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
    return model

class Generator(nn.Module):
    def __init__(self,d_model,tgt_vocab):
        super(Generator, self).__init__()
        self.linear=nn.Linear(d_model,tgt_vocab)

    def forward(self,x):
        x=self.linear(x)
        return F.log_softmax(x, dim=-1)
    

class Encoder_Decoder(nn.Module):
    def __init__(self, Encode,Decode,src_embed, tgt_embed, generator):
        super(Encoder_Decoder, self).__init__()
        self.Encode=Encode
        self.Decode=Decode
        self.src_embed=src_embed
        self.tgt_embed=tgt_embed
        self.generator=generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.Encode(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.Decode(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
class Decode(nn.Module):

    def __init__(self,layer,N):
        super(Decode, self).__init__()
        self.layer=replicate(layer,N)
        self.N=N

    def forward(self,X,y,src_mask,tgt_mask):
        for layer in self.layer:
            X=layer(X,y,src_mask,tgt_mask)
        return X
    
class Decoder(nn.Module):
    def __init__(self, size, FeedForward, Self_Multi_Head_Attention,Encoder_Multi_Head_Attention,AddNorm):
        super(Decoder, self).__init__()
        self.self_attn = Self_Multi_Head_Attention
        self.feed_forward = FeedForward
        self.encoder_attention=Encoder_Multi_Head_Attention
        self.AddNorm=AddNorm
        self.size = size

    def forward(self,x,m,src_mask,tgt_mask):
        x=self.AddNorm(x, self.self_attn(x,x,x,tgt_mask))
        x=self.AddNorm(x, self.encoder_attention(x,m,m,src_mask))
        x=self.AddNorm(x, self.feed_forward(x))
        return x   
    
def make_model(src_vocab,tgt_vocab,N=6,d_model=512,h=8,dropout=0.1,d_ff=2048):
    
    model=Encoder_Decoder(
        Encode(Encoder(d_model,FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),AddNorm(d_model,dropout, eps=1e-6)),N),
        Decode(Decoder(d_model, FeedForward(d_model, d_ff, dropout=dropout), MultiHeadAttention(d_model, h),MultiHeadAttention(d_model,h),AddNorm(d_model,dropout, eps=1e-6)),N),
        nn.Sequential(Embedding(d_model,src_vocab),Positional_Encoding(d_model)),
        nn.Sequential(Embedding(d_model,tgt_vocab),Positional_Encoding(d_model)),
        Generator(d_model,tgt_vocab)
    )
    for p in model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
    return model

In [18]:
test_model = make_model(11, 11, 2)
for name, param in test_model.named_parameters():
    print(f'Name: {name}, Size: {param.size()} ,f"Requires Grad: {param.requires_grad}"')

Name: Encode.layer.0.self_attn.query.weight, Size: torch.Size([512, 512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.query.bias, Size: torch.Size([512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.key.weight, Size: torch.Size([512, 512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.key.bias, Size: torch.Size([512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.value.weight, Size: torch.Size([512, 512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.value.bias, Size: torch.Size([512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.fc_out.weight, Size: torch.Size([512, 512]) ,f"Requires Grad: True"
Name: Encode.layer.0.self_attn.fc_out.bias, Size: torch.Size([512]) ,f"Requires Grad: True"
Name: Encode.layer.0.feed_forward.w_1.weight, Size: torch.Size([2048, 512]) ,f"Requires Grad: True"
Name: Encode.layer.0.feed_forward.w_1.bias, Size: torch.Size([2048]) ,f"Requires Grad: True"
Name: Encode.layer.0.feed_forward.w_2.weight, Size: t

In [17]:
class Batch:
    def __init__(self,src,tgt,pad=2):
        self.src=src
        self.src_mask=(src!=pad).unsqueeze(-2)
        self.tgt=tgt[:,:-1]
        self.tgt_y=tgt[:,1:]
        self.tgt_mask=self.make_std_mask(self.tgt,pad)

    @staticmethod
    def make_std_mask(tgt,pad):
        tgt_mask=(tgt!=pad).unsqueeze(-2)
        tgt_mask=tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
        return tgt_mask  

In [19]:
def data_gen(V,batch_size,n_batches):
    
    for i in range(n_batches):
        # Get the data and labels for the current batch
        data=torch.randint(1,V,size=(batch_size,10))
        data[:,0]=1
        src=data.requires_grad_(False).clone().detach()
        tgt=data.requires_grad_(False).clone().detach()
        yield Batch(src,tgt,pad=0)

In [44]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [65]:
# test_model = make_model(11, 11, 2)
# for batch in data_gen(11,1,1):
#     out=test_model.forward(batch.src,batch.tgt,batch.src_mask,batch.tgt_mask)

# out=test_model.generator(out)


for batch in data_gen(11,1,1):
    memory = test_model.encode(batch.src, batch.src_mask)
    ys = torch.zeros(1, 1).fill_(0).type_as(batch.src.data)
    for i in range(10 - 1):
        out = test_model.decode(
            memory, batch.src_mask, ys, subsequent_mask(ys.size(1)).type_as(batch.src.data)
        )
        print(out.shape)
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(batch.src.data).fill_(next_word)], dim=1
        )
print(batch.src)
print(ys)

torch.Size([1, 1, 512])
torch.Size([1, 2, 512])
torch.Size([1, 3, 512])
torch.Size([1, 4, 512])
torch.Size([1, 5, 512])
torch.Size([1, 6, 512])
torch.Size([1, 7, 512])
torch.Size([1, 8, 512])
torch.Size([1, 9, 512])
tensor([[ 1,  4,  1,  1,  6,  3,  4,  4,  3, 10]])
tensor([[0, 2, 2, 2, 2, 2, 2, 2, 2, 2]])


In [66]:
test_model = make_model(11, 11, 2)
for batch in data_gen(11,1,1):
    out=test_model.forward(batch.src,batch.tgt,batch.src_mask,batch.tgt_mask)

out=test_model.generator(out)
criterion=LabelSmoothing(11,0,0.0)
criterion(out.contiguous().view(-1,out.size(-1)),batch.tgt_y.contiguous().view(-1))

tensor(23.5780, grad_fn=<SumBackward0>)

In [70]:
import numpy as np
numpy_array = out.detach().numpy()

# Save the numpy array to a CSV file
np.savetxt('tensor.csv', numpy_array[0], delimiter=',')

In [78]:
def rate(step_num,d_model,factor,warmup_steps):
    if step_num==0:
        step_num=1
    return d_model**(-0.5) * min(step_num**(-0.5), step_num * warmup_steps**(-1.5))


In [83]:
from torch.optim.lr_scheduler import LambdaLR
test_model = make_model(11, 11, 2)
criterion=LabelSmoothing(11,0,0.0)
optimizer = torch.optim.Adam(test_model.parameters(), lr=1)
lr_lambda = lambda step: rate(step,256,1,4000)

# Create a LambdaLR scheduler
scheduler = LambdaLR(optimizer, lr_lambda)

for epoch in range(20):
    for i,batch in enumerate(data_gen(11,20,30)):  # Assume data_loader is an iterator that provides batches of data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Get the data and labels
        
        # Forward pass
        out = test_model.forward(batch.src,batch.tgt,batch.src_mask,batch.tgt_mask)
        out=test_model.generator(out)
        # Compute loss
        loss = criterion(out.contiguous().view(-1,out.size(-1)),batch.tgt_y.contiguous().view(-1))
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Step the learning rate scheduler
        scheduler.step()

    print(epoch)

RuntimeError: The size of tensor a (10) must match the size of tensor b (20) at non-singleton dimension 1

In [364]:
for i in range(50):
    data=torch.randint(1,11,size=(50,10))
    data[:,0]=1
    src=data.requires_grad_(False).clone().detach()
    tgt=data.requires_grad_(False).clone().detach()

In [374]:
import torch
import torch.nn as nn

# Define the loss function with reduction set to 'sum'
loss_function = nn.KLDivLoss(reduction='sum')

# Define two probability distributions (as PyTorch tensors)
# Input (log probabilities)
input = torch.tensor([[0.2, 0.7, 0.1], [0.9, 0.05, 0.05]], dtype=torch.float).log()
# Target (probabilities)
target = torch.tensor([[0.1, 0.8, 0.1], [0.5, 0.2, 0.3]], dtype=torch.float)

# Compute the loss
loss = loss_function(input, target)
print(loss)

tensor(0.5584)


In [398]:
label smoothing
size=6
padding_idx=0
smoothing=0.4
predict=torch.FloatTensor([[0,0.2,0.7,0.1,0],
                           [0,0.2,0.7,0.1,0],
                           [0,0.2,0.7,0.1,0],
                           [0,0.2,0.7,0.1,0],
                           [0,0.2,0.7,0.1,0],
                           [0,0.2,0.7,0.1,0]])
target=torch.LongTensor([2,1,0,3,3,4])
true_dist = predict.log().data.clone()
true_dist.fill_(smoothing / (size - 2))
true_dist.scatter_(1, target.data.unsqueeze(1), 0.6)
true_dist[:, padding_idx] = 0
mask = torch.nonzero(target.data == padding_idx)
if mask.dim() > 0:
    true_dist.index_fill_(0, mask.squeeze(), 0.0)
s=loss_function(predict.log(), true_dist.clone().detach())
s

tensor(nan)

In [394]:



import pandas as pd
crit = LabelSmoothing(5, 0, 0.4)
predict = torch.FloatTensor(
    [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
    ]
)
crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
LS_data = pd.concat(
    [
        pd.DataFrame(
            {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
            }
        )
        for y in range(5)
        for x in range(5)
    ]
)


# alt.Chart(LS_data)
# .mark_rect(color="Blue", opacity=1)
# .properties(height=200, width=200)
# .encode(
#     alt.X("columns:O", title=None),
#     alt.Y("rows:O", title=None),
#     alt.Color(
#         "target distribution:Q", scale=alt.Scale(scheme="viridis")
#     ),
# )
# .interactive()



In [397]:
crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))

tensor(nan)

In [ ]:
for epoch in range(num_epochs):
    for batch in data_loader:  # Assume data_loader is an iterator that provides batches of data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Get the data and labels
        inputs, labels = batch
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Step the learning rate scheduler
        scheduler.step()
        

In [13]:

for name, param in dummy_mode.named_parameters():
    print(f'Name: {name}, Size: {param.size()}')

Name: weight, Size: torch.Size([1, 1])
Name: bias, Size: torch.Size([1])


In [15]:
dummy_mode.weight.data

tensor([[0.0266]])

In [17]:
from torch.optim.lr_scheduler import LambdaLR
import torch.nn as nn
import torch
dummy_mode=nn.Linear(1,1)
learning_rate=[]
optimizer = torch.optim.Adam(dummy_mode.parameters(), lr=1)

lr_lambda = lambda step: rate(step,256,1,4000)

# Create a LambdaLR scheduler
scheduler = LambdaLR(optimizer, lr_lambda)
tem=[]
for epoch in range(20):
    # (Here you would normally perform the training steps)
    optimizer.zero_grad()
    # Step the scheduler to adjust the learning rate
    tem.append(optimizer.param_groups[0]["lr"])
    print(dummy_mode.weight.data)
    optimizer.step()
    scheduler.step()


tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])
tensor([[-0.5570]])


In [10]:
filename = "output2.csv"
import csv
# Writing to csv file
with open(filename, 'w', newline='') as csvfile:
    # Create a csv writer object
    csvwriter = csv.writer(csvfile)
    
    # Write each row to the csv file
    for row in tem:
        csvwriter.writerow([row])